In [1]:
# pip install pymongo

In [28]:
import redis
import datetime
import pymongo
import json
import sys
import time
from time import sleep
import os
from datetime import datetime
from datetime import timedelta
import pandas as pd
import math
import numpy as np
import scipy
from scipy.stats import norm
from scipy import sqrt, log, exp
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame

N = norm.cdf
r=redis.Redis(host='localhost', port=6379,db=6, decode_responses= True)

In [29]:
def black_scholes_call(S, X, T, r, sigma):
    d1 = (math.log(S/X) + (r + sigma**2/2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call_value = S * norm.cdf(d1) - X * math.exp(-r * T) * norm.cdf(d2)
    call_value=float(call_value)
    call_value=round(call_value,2)
    return (call_value)

In [30]:
def d1(S, X, T, r, sigma):
    return (math.log(S/X) + (r + sigma**2/2) * T) / (sigma * math.sqrt(T))

In [31]:
def black_scholes_put(S, X, T, r, sigma):
    d1 = (np.log(S/X) + (r + sigma**2/2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_value=(X * np.exp(-r*T) * N(-d2) - S * N(-d1))
    put_value=float(put_value)
    put_value=round(put_value,2)
    return put_value

In [32]:
# black_scholes_put(43867, 43900, t1, 0.1, sigma)

In [33]:
# d1(43867, 43900, t1, 0.1, sigma)

In [34]:
def get_expiration_time_remaining():
    myclient = pymongo.MongoClient("mongodb://192.168.1.110:27017")
    mydb = myclient["Details"]
    coll = mydb['expiries']
    temp = coll.find_one()
    current_exp = datetime.strptime(temp["current"], "%Y-%m-%d")
    current_exp=current_exp+timedelta(hours=15.5)
    days=((current_exp-datetime.today()).days)
    hours=((current_exp-datetime.today()).seconds/(60*60))
    days=days+hours/24
    return days/365
def get_expiration_time_remainings():
    myclient = pymongo.MongoClient("mongodb://192.168.1.110:27017")
    mydb = myclient["Details"]
    coll = mydb['expiries']
    temp = coll.find_one()
    current_exp = datetime.strptime(temp["next"], "%Y-%m-%d")
    current_exp=current_exp+timedelta(hours=15.5)
    days=((current_exp-datetime.today()).days)
    hours=((current_exp-datetime.today()).seconds/(60*60))
    days=days+hours/24
    return days/365


In [35]:
current_time_expiry = get_expiration_time_remaining()
far_time_expiry = get_expiration_time_remainings()

t1 = current_time_expiry
t2 = far_time_expiry

In [36]:
# bs_call(43862,45000,t1,0.10,sigma)

In [37]:
#############################################################################################
#Finding the sigma from the one year data of banknifty

In [38]:
import yfinance as yf


In [39]:
stock = '^NSEBANK'
today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

df = yf.download(stock,start=one_year_ago, end =today )

[*********************100%***********************]  1 of 1 completed


In [40]:
df = yf.download(stock,start=one_year_ago, end =today )
stock = '^NSEBANK'
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))

[*********************100%***********************]  1 of 1 completed


In [41]:
from xlwings.utils import rgb_to_int
def colour(a):
    if a>=20:
        return (255, 0, 0)
    elif a<=-20:
        return (0,0,255)
    else:
        return (0,0,0)

In [42]:
stock = '^NSEBANK'
today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = india_vix/100

In [ ]:
################################   final code   #################################

from pymongo import MongoClient
import sys
from datetime import timedelta
import datetime
from pprint import pprint
from datetime import datetime
import redis
import json
import pandas as pd
import numpy as np
import time
import threading
from time import sleep
import xlwings as xw
# sys.path.insert(1, r'C:\Users\BPS2\Imported_files')
from AttachExpiry import get_expiry_names
from Greeks import implied_volatility_put
from Greeks import implied_volatility_call


flag=1
r=0.1
list_india_vix=[11.02,11,11.01,11.07,11.01,10.99,10.95,10.95,10.93,10.93,10.89,10.84,10.82,10.79,10.78,10.79,10.81,10.82,10.77,10.81]
p=0
RedisClientInd = redis.Redis(host='192.168.1.110', port='6379', db='7',decode_responses=True)
RedisClientInd_Fut = redis.Redis(host='192.168.1.110', port='6379', db='5',decode_responses=True)
RedisClientInd_Vix = redis.Redis(host='192.168.1.110', port='6379', db='7',decode_responses=True)
RedisClientInd_curr = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
#RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
RedisClientInd_far = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
#RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)

sym_name_vix = "INDIA_VIX"
sym_name = "BankNifty"
sym_name_Fut = "BANKNIFTY23JULFUT"

t=1
while t:
    try:
        
        current_time_expiry = get_expiration_time_remaining()
        far_time_expiry = get_expiration_time_remainings()
        t1 = current_time_expiry
        t2 = far_time_expiry
        T=get_expiration_time_remaining()
        
        
        current_expiry_date=get_expiry_names("BANKNIFTY","OPT")[0]
        far_expiry_date=get_expiry_names("BANKNIFTY","OPT")[1]

        sym_name = "BankNifty"
        index_data=json.loads(RedisClientInd.get(sym_name))
        price_index_live = float(index_data['Touchline']['LastTradedPrice'])
        S=price_index_live
        
        
        index_data_Fut=json.loads(RedisClientInd_Fut.get(sym_name_Fut))
        price_index_live_Fut = float(index_data_Fut['Touchline']['LastTradedPrice'])  

        index_data_vix=json.loads(RedisClientInd_Vix.get(sym_name_vix))
        price_index_live_vix = float(index_data_vix['Touchline']['LastTradedPrice'])  

        india_vix = price_index_live_vix
        #############   india-vix graph   ############
        
        p+=1
        if p==5:
            list_india_vix.pop(0)
            list_india_vix.extend([india_vix])
            p=0

        #read index spot values from data
        index_spot=price_index_live

        #rounding off index spot
        rem=index_spot%100
        if rem>=50:
            strike_spot=int(index_spot/100)*100+100

        else:
            strike_spot=int(index_spot/100)*100
        

        strike_spot-=1400


        index_futures=price_index_live_Fut

        dict={
            "index spot":index_spot,
            "index futures":index_futures,
            "strikes":{

                }
        }
  
        s=dict['strikes']

        i=0
        while i<27:
        #set a variable to fetch near and far expiry value for the corresponding strike
           
            s[strike_spot]=[]
           

            #creating new symbol name to fetch expiry values
            add_strike=str(strike_spot)
            new_sym_name_current_call=str(current_expiry_date)+add_strike+"CE"
            new_sym_name_far_call=str(far_expiry_date)+add_strike+"CE"
            new_sym_name_current_put=str(current_expiry_date)+add_strike+"PE"
            new_sym_name_far_put=str(far_expiry_date)+add_strike+"PE"   
           
            index_data_curr=json.loads(RedisClientInd_curr.get(new_sym_name_current_call))
            curr_expiryvalue_call = float(index_data_curr['Touchline']['LastTradedPrice']) 
            index_data_curr=json.loads(RedisClientInd_curr.get(new_sym_name_current_put))
            curr_expiryvalue_put = float(index_data_curr['Touchline']['LastTradedPrice'])

            index_data_far=json.loads(RedisClientInd_far.get(new_sym_name_far_call))
            far_expiryvalue_call = float(index_data_far['Touchline']['LastTradedPrice'])   
            index_data_far=json.loads(RedisClientInd_far.get(new_sym_name_far_put))
            far_expiryvalue_put = float(index_data_far['Touchline']['LastTradedPrice'])
            try:
                iv_call_currrent = round(implied_volatility_call(S, strike_spot, T, r, curr_expiryvalue_call, sigma_estimate=0.2, tol=1e-5, max_iter=20),2)
            
            except Exception as e:
                print(e)
                iv_call_currrent=None
                pass
            
            try:
                iv_put_current = round(implied_volatility_put(S, strike_spot, T*365, r, curr_expiryvalue_put, sigma_estimate=0.2, tol=1e-5, max_iter=20),2)
               
            
            except:
                iv_put_current=None
                pass

            s[strike_spot].append(curr_expiryvalue_call)
            s[strike_spot].append(far_expiryvalue_call)
            s[strike_spot].append(curr_expiryvalue_put)
            s[strike_spot].append(far_expiryvalue_put)
#             iv_call_currrent=0
#             iv_put_current=0
            s[strike_spot].append(iv_call_currrent)
            s[strike_spot].append(iv_put_current)
            strike_spot+=100

            i+=1
       
        with open("sample.json","w")as outfile:
            json.dump(dict,outfile)

       # print(dict)

        ws=xw.Book('Excel1.xlsx').sheets("BANKNIFTY")

        ########################################################################



        stock = '^NSEBANK'
        today = datetime.now()
        one_year_ago = today.replace(year=today.year-1)

        df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

        sigma = india_vix/100

        ##########################################################################

        #importing json file
        with open('sample.json','r')as openfile:
            json_object=json.load(openfile)
        index_spot=json_object.get('index spot')
        index_futures=json_object.get('index futures')

        #placing value in excel
        ws.range("A1").value='BANKNIFTY'
        ws.range("A2").value=index_spot

        ws.range("B1").value='BANKNIFTYFUTURES'
        ws.range("B2").value=index_futures

        ws.range("C1").value='INDIA_VIX'
        ws.range("C2").value=india_vix

        ws.range("A5").value='Far Expiry'
        ws.range('B5').value = 'BS Far Expiry'
        ws.range("C5").value='Current Expiry'
        ws.range("D5").value = 'BS Current Expiry'
        ws.range("E5").value='STRIKES'
        ws.range('F5').value='BS Current Expiry'
        ws.range('G5').value = 'Current Expiry'
        ws.range('H5').value = 'BS Far Expiry'
        ws.range('I5').value = 'Far Expiry'
        ws.range('J5').value = 'IV call'
        ws.range('K5').value = 'IV put'
        ws.range('U5').value = 'VIX'
        strikerates=json_object.get('strikes')
        rownum=6
        for keys,values in strikerates.items():
            ws.range('E'+str(rownum)).value=keys
            ws.range('C'+str(rownum)).value=values[0]
            ws.range('A'+str(rownum)).value=values[1]
            ws.range('J'+str(rownum)).value=values[4]
            ws.range('K'+str(rownum)).value=values[5]
            
            ws.range('B'+str(rownum)).value=black_scholes_call(float(index_spot),float(keys),t2,0.1,sigma)
    #         c = ws.range('C'+str(rownum)).value - ws.range('D'+str(rownum)).value
    #         ws.range('C'+str(rownum)).api.Font.Color = rgb_to_int(colour(c))

            ws.range('D'+str(rownum)).value=black_scholes_call(float(index_spot),float(keys),t1,0.1,sigma)
            d = ws.range('A'+str(rownum)).value - ws.range('B'+str(rownum)).value
            ws.range('A'+str(rownum)).api.Font.Color = rgb_to_int(colour(d))
            c = ws.range('C'+str(rownum)).value - ws.range('D'+str(rownum)).value
            ws.range('C'+str(rownum)).api.Font.Color = rgb_to_int(colour(c))

            ws.range('F'+str(rownum)).value=black_scholes_put(float(index_spot),float(keys),t1,0.1,sigma)
            ws.range('G'+str(rownum)).value= values[2]
            b = ws.range('G'+str(rownum)).value - ws.range('F'+str(rownum)).value
            ws.range('G'+str(rownum)).api.Font.Color = rgb_to_int(colour(b))

            ws.range('H'+str(rownum)).value= black_scholes_put(float(index_spot),float(keys),t2,0.1,sigma)
            ws.range('I'+str(rownum)).value= values[3]
            a = ws.range('I'+str(rownum)).value - ws.range('H'+str(rownum)).value
            ws.range('I'+str(rownum)).api.Font.Color = rgb_to_int(colour(a))

            #for india-vix
            if rownum<=25:
                ws.range('U'+str(rownum)).value=list_india_vix[rownum-6]
                
            rownum+=1

        #outputting the table
        #print("Data :\n",ws.range("A1").expand().value)

        sleep(1)
        
    except Exception as e:
        print(e)
        print('data error')

#3.printing data in excel



44905.8 43500 0.0006301052765093862 0.1 1425.0
hello
44905.8 43600 0.0006301052765093862 0.1 1302.95
hello
44905.8 43700 0.0006301052765093862 0.1 1219.75
hello
44905.8 43800 0.0006301052765093862 0.1 1107.25
hello
44905.8 43900 0.0006301052765093862 0.1 1029.0
hello
44905.8 44000 0.0006301052765093862 0.1 925.15
hello
44905.8 44100 0.0006301052765093862 0.1 825.0
hello
44905.8 44200 0.0006301052765093862 0.1 723.35
hello
44905.8 44300 0.0006301052765093862 0.1 621.85
hello
44905.8 44400 0.0006301052765093862 0.1 521.9
hello
44905.8 44500 0.0006301052765093862 0.1 427.6
hello
44905.8 44600 0.0006301052765093862 0.1 335.85
hello
44905.8 44700 0.0006301052765093862 0.1 252.0
hello
44905.8 44800 0.0006301052765093862 0.1 177.6
hello
44905.8 44900 0.0006301052765093862 0.1 116.45
hello
44905.8 45000 0.0006301052765093862 0.1 72.7
hello
44905.8 45100 0.0006301052765093862 0.1 42.7
hello
44905.8 45200 0.0006301052765093862 0.1 24.1
hello
44905.8 45300 0.0006301052765093862 0.1 13.55
hello
44

C:\Users\admin\BPS2\Greeks.py:53: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = sigma + diff/vega
C:\Users\admin\BPS2\Greeks.py:25: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (math.log(S/X) + (r + sigma**2/2) * T) / (sigma * math.sqrt(T))
C:\Users\admin\BPS2\Greeks.py:50: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (np.lib.scimath.log(S/X) + (r + sigma**2/2) * T) / (sigma * np.lib.scimath.sqrt(T))


44907.05 43500 0.0006290905631659056 0.1 1418.9
hello
44907.05 43600 0.0006290905631659056 0.1 1302.95
hello
44907.05 43700 0.0006290905631659056 0.1 1219.75
hello
44907.05 43800 0.0006290905631659056 0.1 1107.25
hello
44907.05 43900 0.0006290905631659056 0.1 1029.0
hello
44907.05 44000 0.0006290905631659056 0.1 921.9
hello
44907.05 44100 0.0006290905631659056 0.1 825.0
hello
44907.05 44200 0.0006290905631659056 0.1 722.75
hello
44907.05 44300 0.0006290905631659056 0.1 623.95
hello
44907.05 44400 0.0006290905631659056 0.1 523.5
hello
44907.05 44500 0.0006290905631659056 0.1 429.8
hello
44907.05 44600 0.0006290905631659056 0.1 339.1
hello
44907.05 44700 0.0006290905631659056 0.1 253.3
hello
44907.05 44800 0.0006290905631659056 0.1 178.45
hello
44907.05 44900 0.0006290905631659056 0.1 117.8
hello
44907.05 45000 0.0006290905631659056 0.1 73.65
hello
44907.05 45100 0.0006290905631659056 0.1 42.95
hello
44907.05 45200 0.0006290905631659056 0.1 24.3
hello
44907.05 45300 0.0006290905631659056

C:\Users\admin\BPS2\Greeks.py:25: RuntimeWarning: overflow encountered in double_scalars
  d1 = (math.log(S/X) + (r + sigma**2/2) * T) / (sigma * math.sqrt(T))
C:\Users\admin\BPS2\Greeks.py:50: RuntimeWarning: overflow encountered in double_scalars
  d1 = (np.lib.scimath.log(S/X) + (r + sigma**2/2) * T) / (sigma * np.lib.scimath.sqrt(T))


44923.55 43500 0.0006273465246067986 0.1 1418.9
hello
44923.55 43600 0.0006273465246067986 0.1 1302.95
hello
44923.55 43700 0.0006273465246067986 0.1 1219.75
hello
44923.55 43800 0.0006273465246067986 0.1 1107.25
hello
44923.55 43900 0.0006273465246067986 0.1 1029.0
hello
44923.55 44000 0.0006273465246067986 0.1 925.4
hello
44923.55 44100 0.0006273465246067986 0.1 819.95
hello
44923.55 44200 0.0006273465246067986 0.1 725.85
hello
44923.55 44300 0.0006273465246067986 0.1 624.0
hello
44923.55 44400 0.0006273465246067986 0.1 529.75
hello
44923.55 44500 0.0006273465246067986 0.1 434.85
hello
44923.55 44600 0.0006273465246067986 0.1 341.75
hello
44923.55 44700 0.0006273465246067986 0.1 255.65
hello
44923.55 44800 0.0006273465246067986 0.1 180.6
hello
44923.55 44900 0.0006273465246067986 0.1 119.4
hello
44923.55 45000 0.0006273465246067986 0.1 74.45
hello
44923.55 45100 0.0006273465246067986 0.1 43.65
hello
44923.55 45200 0.0006273465246067986 0.1 24.5
hello
44923.55 45300 0.0006273465246067

...

44937.4 43500 0.0006133307965499746 0.1 1437.65
hello
44937.4 43600 0.0006133307965499746 0.1 1340.05
hello
44937.4 43700 0.0006133307965499746 0.1 1239.05
hello
44937.4 43800 0.0006133307965499746 0.1 1148.7
hello
44937.4 43900 0.0006133307965499746 0.1 1033.25
hello
44937.4 44000 0.0006133307965499746 0.1 937.1
hello
44937.4 44100 0.0006133307965499746 0.1 830.0
hello
44937.4 44200 0.0006133307965499746 0.1 736.0
hello
44937.4 44300 0.0006133307965499746 0.1 640.4
hello
44937.4 44400 0.0006133307965499746 0.1 543.0
hello
44937.4 44500 0.0006133307965499746 0.1 448.25
hello
44937.4 44600 0.0006133307965499746 0.1 349.05
hello
44937.4 44700 0.0006133307965499746 0.1 269.85
hello
44937.4 44800 0.0006133307965499746 0.1 189.0
hello
44937.4 44900 0.0006133307965499746 0.1 127.85
hello
44937.4 45000 0.0006133307965499746 0.1 80.65
hello
44937.4 45100 0.0006133307965499746 0.1 45.85
hello
44937.4 45200 0.0006133307965499746 0.1 25.85
hello
44937.4 45300 0.0006133307965499746 0.1 14.25
hello

44976.25 46200 0.000608288939624556 0.1 2.4
hello
44973.9 43600 0.0006073059360730594 0.1 1364.5
hello
44973.9 43700 0.0006073059360730594 0.1 1239.05
hello
44973.9 43800 0.0006073059360730594 0.1 1170.0
hello
44973.9 43900 0.0006073059360730594 0.1 1056.6
hello
44973.9 44000 0.0006073059360730594 0.1 961.55
hello
44973.9 44100 0.0006073059360730594 0.1 855.5
hello
44973.9 44200 0.0006073059360730594 0.1 757.5
hello
44973.9 44300 0.0006073059360730594 0.1 662.0
hello
44973.9 44400 0.0006073059360730594 0.1 563.4
hello
44973.9 44500 0.0006073059360730594 0.1 465.55
hello
44973.9 44600 0.0006073059360730594 0.1 376.4
hello
44973.9 44700 0.0006073059360730594 0.1 285.9
hello
44973.9 44800 0.0006073059360730594 0.1 208.9
hello
44973.9 44900 0.0006073059360730594 0.1 139.45
hello
44973.9 45000 0.0006073059360730594 0.1 90.7
hello
44973.9 45100 0.0006073059360730594 0.1 52.75
hello
44973.9 45200 0.0006073059360730594 0.1 30.05
hello
44973.9 45300 0.0006073059360730594 0.1 16.3
hello
44973.9 

44960.55 43600 0.0006012176560121766 0.1 1364.5
hello
44960.55 43700 0.0006012176560121766 0.1 1239.05
hello
44960.55 43800 0.0006012176560121766 0.1 1170.0
hello
44960.55 43900 0.0006012176560121766 0.1 1047.75
hello
44960.55 44000 0.0006012176560121766 0.1 961.15
hello
44960.55 44100 0.0006012176560121766 0.1 855.45
hello
44960.55 44200 0.0006012176560121766 0.1 757.45
hello
44960.55 44300 0.0006012176560121766 0.1 658.75
hello
44960.55 44400 0.0006012176560121766 0.1 561.25
hello
44960.55 44500 0.0006012176560121766 0.1 466.0
hello
44960.55 44600 0.0006012176560121766 0.1 368.1
hello
44960.55 44700 0.0006012176560121766 0.1 284.85
hello
44960.55 44800 0.0006012176560121766 0.1 206.75
hello
44960.55 44900 0.0006012176560121766 0.1 136.3
hello
44960.55 45000 0.0006012176560121766 0.1 88.55
hello
44960.55 45100 0.0006012176560121766 0.1 51.2
hello
44960.55 45200 0.0006012176560121766 0.1 29.05
hello
44960.55 45300 0.0006012176560121766 0.1 16.0
hello
44960.55 45400 0.000601217656012176

44971.45 43600 0.0005951293759512938 0.1 1353.1
hello
44971.45 43700 0.0005951293759512938 0.1 1253.2
hello
44971.45 43800 0.0005951293759512938 0.1 1155.0
hello
44971.45 43900 0.0005951293759512938 0.1 1054.15
hello
44971.45 44000 0.0005951293759512938 0.1 954.2
hello
44971.45 44100 0.0005951293759512938 0.1 855.0
hello
44971.45 44200 0.0005951293759512938 0.1 754.55
hello
44971.45 44300 0.0005951293759512938 0.1 655.85
hello
44971.45 44400 0.0005951293759512938 0.1 560.0
hello
44971.45 44500 0.0005951293759512938 0.1 464.4
hello
44971.45 44600 0.0005951293759512938 0.1 370.8
hello
44971.45 44700 0.0005951293759512938 0.1 281.75
hello
44971.45 44800 0.0005951293759512938 0.1 203.4
hello
44971.45 44900 0.0005951293759512938 0.1 138.55
hello
44971.45 45000 0.0005951293759512938 0.1 87.2
hello
44971.45 45100 0.0005951293759512938 0.1 52.4
hello
44971.45 45200 0.0005951293759512938 0.1 29.25
hello
44971.45 45300 0.0005951293759512938 0.1 16.0
hello
44971.45 45400 0.0005951293759512938 0.1

44944.2 46100 0.0005900875190258752 0.1 2.55
hello
44942.3 43500 0.0005890728056823948 0.1 1437.25
hello
44942.3 43600 0.0005890728056823948 0.1 1346.6
hello
44942.3 43700 0.0005890728056823948 0.1 1231.7
hello
44942.3 43800 0.0005890728056823948 0.1 1144.7
hello
44942.3 43900 0.0005890728056823948 0.1 1025.0
hello
44942.3 44000 0.0005890728056823948 0.1 936.4
hello
44942.3 44100 0.0005890728056823948 0.1 832.2
hello
44942.3 44200 0.0005890728056823948 0.1 726.45
hello
44942.3 44300 0.0005890728056823948 0.1 636.3
hello
44942.3 44400 0.0005890728056823948 0.1 538.2
hello
44942.3 44500 0.0005890728056823948 0.1 441.95
hello
44942.3 44600 0.0005890728056823948 0.1 350.7
hello
44942.3 44700 0.0005890728056823948 0.1 263.0
hello
44942.3 44800 0.0005890728056823948 0.1 185.3
hello
44942.3 44900 0.0005890728056823948 0.1 123.15
hello
44942.3 45000 0.0005890728056823948 0.1 77.2
hello
44942.3 45100 0.0005890728056823948 0.1 44.55
hello
44942.3 45200 0.0005890728056823948 0.1 24.85
hello
44942

In [ ]:
iv_put_current = round(implied_volatility_put(45072, 45100, 2/365, 0.1, curr_expiryvalue_put, sigma_estimate=0.2, tol=1e-5, max_iter=20),2)
print("print(S,X,T,r,curr_expiryvalue_put)",iv_put_current)

In [75]:
type(strike_spot)

int

In [76]:
strike_spot

46100

In [72]:
try:
    p=implied_volatility_call(S, 45100, T, 0.1, 156, sigma_estimate=0.2, tol=1e-5, max_iter=20)
    iv_call_currrent = round(implied_volatility_call(S, 45100, T, 0.1, 156, sigma_estimate=0.2, tol=1e-5, max_iter=20),2)
    print(iv_call_currrent)
except Exception as e:
    print(e)
    
    
print(p)

0.12
0.11875287045364283


In [61]:
iv_call_currrent

nan

In [ ]:
T